In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import json
import folium

## Binning courses

In [3]:
# Reading the dataframe
CD = pd.read_excel("DataFrame.xlsx", dtype={'Course code': str})

In [4]:
 #Correcting some errors
CD.loc[115,"Course name"] = "Staging co-creation and creativity"
CD.loc[CD["Course name"] == "Facilitating Innovation in Multidisciplinary teams", "Course name"] = "Facilitating Innovation in Multidisciplinary Teams"

In [5]:
# Adding a column for learning categories
CD[["Category"]] = ""

In [6]:
CD

,Course code,Course name,Semester,Level,ECTS,Course type,Passed DI students,Passed students,Average grade,Placement,URL,Eval_1.1,Eval_1.2,Eval_1.3,Eval_1.4,Eval_1.5,Eval_2.1,Category
0,02266,User Experience Engineering,Fall 2019,MSc,5,Semi-Elective,14,118.0,9.7,January,https://kurser.dtu.dk/course/02266,NaN,NaN,NaN,NaN,NaN,NaN,
1,02633,Introduction to programming and data processing,Fall 2019,BSc,5,Elective,<=5,220.0,6.2,January,https://kurser.dtu.dk/course/02633,NaN,NaN,NaN,NaN,NaN,NaN,
2,02266,User Experience Engineering,Fall 2020,MSc,5,Semi-Elective,13,122.0,10.1,January,https://kurser.dtu.dk/course/02266,4.2,4.7,4.5,4.6,4.4,3.7,
3,02393,Programming in C++,Fall 2020,MSc,5,Elective,<=5,187.0,Pass/Non-Pass,Fall,https://kurser.dtu.dk/course/02393,4.0,4.4,4.1,3.8,4.3,2.9,
4,02450,Introduction to Machine Learning and Data Mining,Fall 2020,BSc,5,Elective,<=5,550.0,5.5,Fall,https://kurser.dtu.dk/course/02450,4.1,4.0,3.5,3.7,3.7,3.7,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,42505,Facilitating Innovation in Multidisciplinary T...,Spring 2024,MSc,5,Mandatory,18,71.0,Pass/Non-Pass,August,https://kurser.dtu.dk/course/42505,3.0,3.4,2.8,4.1,3.6,3.2,
407,42543,Management of organizational change,Spring 2024,MSc,5,Elective,28,323.0,8.5,Spring,https://kurser.dtu.dk/course/42543,3.0,3.7,2.7,3.3,2.9,2.9,
408,46797,"Formula Student Electric: implementation, test...",Spring 2024,BSc,5,Elective,<=5,6.0,Pass/Non-Pass,Spring,https://kurser.dtu.dk/course/46797,4.7,4.7,5.0,4.7,4.3,4.0,
409,63851,Project Management,Spring 2024,MSc,5,Semi-Elective,13,156.0,9,August,https://kurser.dtu.dk/course/63851,4.4,4.4,4.1,4.3,4.0,3.7,


In [7]:
pd.set_option('display.max_rows', None) #df viewing options

In [8]:
managementAndBuisness=["42433", '42085', '42383', '42388', '42389', '42421','25352','38106','38113','42009','42196','42107','42438','02431','42415','42105','42543','38301','38001','38302','42016']
medtec=['22435', '27002', '22475','22449','25352','23531','22284']
manufacturingAndMaterials=['41744','41653','41733','10750','28244','41347','11909','31300','41501','41514','41665','41511','42370','10862','41342','41528','41735','46797','12701','41659']
designforpeople=['42071', '22700','42554','02805','42081', "02266"]
supplychain=['41078','42371','02431']
productdesign=['10316', '42071','34052','41742','47203','10750','12250','41078','41618','41031','41342','46797']
systemsengineering=['41078','42083','42371','02431','42370','42270','41061','42543']
circularitySustainability=['28870', '12132', '12139','12104', '25301','47202','12111','12250','30755','22281','42378','42391','12143','12205','12211','12773','12953','23532','27510','38001','12701']
SmartProducts=["02830", '31383', '34338','34540','02162','34367','34723','02564','02160','31375','31392','41028','02105']
tecnicalCompetences=["88383", "02633", "02450", '10316', '27002', '28213','34052','02805','02162','02402','28244','41347','02003','41525','41822','02403','02102','10420','22284','28864','41522','41571','02411','01035','02105','4165','415289','10603','41659']

category_lists = {
    'managementAndBuisness': list(map(str, managementAndBuisness)),
    'medtec': list(map(str, medtec)),
    'manufacturingAndMaterials': list(map(str, manufacturingAndMaterials)),
    'designforpeople': list(map(str, designforpeople)),
    'supplychain': list(map(str, supplychain)),
    'productdesign': list(map(str, productdesign)),
    'systemsengineering': list(map(str, systemsengineering)),
    'circularitySustainability': list(map(str, circularitySustainability)),
    'SmartProducts': list(map(str, SmartProducts)),
    'tecnicalCompetences': list(map(str, tecnicalCompetences)) # Not just programming?
}

def find_categories(course):
    return [name for name, codes in category_lists.items() if course in codes]

CD['Category'] = CD['Course code'].apply(find_categories)
CD['Category'] = CD['Category'].apply(lambda x: ', '.join(x))

In [9]:
# If a course has the same name as another that already has a category assigned, the same category will be assigned
# Create a mapping from course name to category where Category is not empty
course_to_category = CD[CD["Category"] != ""].drop_duplicates(subset="Course name").set_index("Course name")["Category"]

# Use the mapping to fill missing categories based on course name
CD["Category"] = CD["Category"].replace("", None)  # replace empty string with actual NaN
CD["Category"] = CD["Category"].fillna(CD["Course name"].map(course_to_category))

#### Til Alex

In [11]:
# Viser hvilke der mangler ud fra category navn
df_missing_category = CD[CD["Category"].isna()]
df_missing_category_unique = df_missing_category.drop_duplicates(subset="Course name", keep="first").reset_index(drop=True)
df_missing_category_unique

,Course code,Course name,Semester,Level,ECTS,Course type,Passed DI students,Passed students,Average grade,Placement,URL,Eval_1.1,Eval_1.2,Eval_1.3,Eval_1.4,Eval_1.5,Eval_2.1,Category
0,02393,Programming in C++,Fall 2020,MSc,5,Elective,<=5,187.0,Pass/Non-Pass,Fall,https://kurser.dtu.dk/course/02393,4.0,4.4,4.1,3.8,4.3,2.9,NaN
1,02809,UX Design Prototyping,Fall 2020,MSc,5,Mandatory,22,180.0,9.5,Fall,https://kurser.dtu.dk/course/02809,3.8,4.2,3.7,4.1,3.6,3.7,NaN
2,10605,History of Technology,Fall 2020,MSc,5,Elective,<=5,16.0,10,Fall,https://kurser.dtu.dk/course/10605,4.5,4.8,4.6,4.1,4.4,2.4,NaN
3,31385,Autonomous robot systems,Fall 2020,MSc,5,Elective,<=5,95.0,Pass/Non-Pass,January,https://kurser.dtu.dk/course/31385,3.5,4.0,3.8,3.3,3.3,3.0,NaN
4,34365,IoT Prototyping,Fall 2020,MSc,5,Semi-Elective,<=5,20.0,Pass/Non-Pass,Fall,https://kurser.dtu.dk/course/34365,3.8,4.0,4.0,3.7,3.6,2.9,NaN
5,41051,Product life and environmental issues,Fall 2020,BSc,5,Elective,<=5,133.0,8.2,Fall,https://kurser.dtu.dk/course/41051,3.9,4.0,3.4,3.7,3.2,3.4,NaN
6,41073,Development and operation of product/service-s...,Fall 2020,MSc,10,Semi-Elective,33,52.0,9.3,Fall,https://kurser.dtu.dk/course/41073,4.6,4.7,4.4,4.3,4.3,3.3,NaN
7,41084,Biologically Inspired Design,Fall 2020,MSc,5,Semi-Elective,31,58.0,10.3,January,https://kurser.dtu.dk/course/41084,4.0,4.5,4.0,4.2,3.9,3.0,NaN
8,41628,Conceptualisation,Fall 2020,MSc,10,Elective,<=5,45.0,8.3,Fall,https://kurser.dtu.dk/course/41628,3.3,3.9,3.6,4.6,3.6,3.0,NaN
9,41743,"Micro product design, development and production",Fall 2020,MSc,5,Elective,<=5,19.0,9.9,January,https://kurser.dtu.dk/course/41743,3.8,3.8,3.9,4.5,4.2,4.1,NaN


In [12]:
# Kan give row index på CD hvis du vil kryds tjekke
name = "Sustainability in management" #Ændrer denne her for at krydstjekke
matching_indices = CD.index[CD["Course name"] == name].tolist()
CD.iloc[matching_indices,:]

,Course code,Course name,Semester,Level,ECTS,Course type,Passed DI students,Passed students,Average grade,Placement,URL,Eval_1.1,Eval_1.2,Eval_1.3,Eval_1.4,Eval_1.5,Eval_2.1,Category
22,42351,Sustainability in management,Fall 2020,MSc,5,Elective,9,230.0,9.3,Fall,https://kurser.dtu.dk/course/42351,3.4,3.9,3.2,4.0,3.4,2.8,NaN
69,42351,Sustainability in management,Fall 2021,MSc,5,Elective,29,189.0,9.9,Fall,https://kurser.dtu.dk/course/42351,3.6,4.0,3.4,3.8,3.6,2.9,NaN
93,12771,Sustainability in management,Fall 2022,MSc,5,Elective,18,193.0,Pass/Non-Pass,Fall,https://kurser.dtu.dk/course/12771,3.5,3.9,3.3,4.0,3.8,3.1,NaN


## Evaluations

In [14]:
df_e = CD.copy()

In [15]:
df_e.dropna(subset=['Eval_1.1', 'Eval_1.2', 'Eval_1.3', 'Eval_1.4', 'Eval_1.5'], inplace=True)

In [16]:
semester_order = ["Fall 2024", "Spring 2024",
                 "Fall 2023", "Spring 2023",
                 "Fall 2022", "Spring 2022",
                 "Fall 2021", "Spring 2021",
                 "Fall 2020", "Spring 2020"]
semester_order = pd.Categorical(semester_order, categories=semester_order, ordered=True)

In [17]:
#Sorts out repeating courses by only having the rows of newest passed course
df_e['Semester'] = pd.Categorical(df_e['Semester'], categories=semester_order, ordered=True)
df_e_sorted = df_e.sort_values(by=['Course name', 'Semester'])
df_e = df_e_sorted.drop_duplicates(subset='Course name', keep='first')
df_e = df_e_sorted.drop_duplicates(subset='Course code', keep='first') # sorts out a few errors where the named differed slighly, but the course code is the same

In [19]:
#consolidating evaluations for those that are not time load related, excluding Eval_2.1
df_e["Evaluation"] = df_e[["Eval_1.1", "Eval_1.2", "Eval_1.3", "Eval_1.4", "Eval_1.5"]].mean(axis=1)

C:\Users\johan\AppData\Local\Temp\ipykernel_16508\3444246911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_e["Evaluation"] = df_e[["Eval_1.1", "Eval_1.2", "Eval_1.3", "Eval_1.4", "Eval_1.5"]].mean(axis=1)


In [20]:
df_e = df_e.drop(columns = ["Eval_1.1", "Eval_1.2", "Eval_1.3", "Eval_1.4", "Eval_1.5", "Eval_2.1"])

In [27]:
df_e = df_e.sort_values(by='Evaluation', ascending=True)

In [30]:
df_e["Evaluation"] = round(df_e["Evaluation"], 2)

In [38]:
df_e = df_e.reset_index(drop=True)

In [41]:
from bokeh.models import TapTool, CustomJS, Div
from bokeh.layouts import column
from bokeh.io import output_file, show, save
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, WheelZoomTool
from bokeh.models import Range1d
from bokeh.models import Legend, LegendItem

In [43]:
# BOKEH

# Create ColumnDataSources
all_source = ColumnDataSource(df_e)
managementAndBuisness_source = ColumnDataSource(df_e[df_e['Category'].str.contains('managementAndBuisness', na=False)])
medtec_source = ColumnDataSource(df_e[df_e['Category'].str.contains('medtec', na=False)])
manufacturingAndMaterials_source = ColumnDataSource(df_e[df_e['Category'].str.contains('manufacturingAndMaterials', na=False)])
designforpeople_source = ColumnDataSource(df_e[df_e['Category'].str.contains('designforpeople', na=False)])
supplychain_source = ColumnDataSource(df_e[df_e['Category'].str.contains('supplychain', na=False)])
productdesign_source = ColumnDataSource(df_e[df_e['Category'].str.contains('productdesign', na=False)])
systemsengineering_source = ColumnDataSource(df_e[df_e['Category'].str.contains('systemsengineering', na=False)])
circularitySustainability_source = ColumnDataSource(df_e[df_e['Category'].str.contains('circularitySustainability', na=False)])
SmartProducts_source = ColumnDataSource(df_e[df_e['Category'].str.contains('SmartProducts', na=False)])
tecnicalCompetences_source = ColumnDataSource(df_e[df_e['Category'].str.contains('tecnicalCompetences', na=False)])

# Create a figure with custom axes labels
p = figure(title="Course Evaluation Tool",
           y_axis_label='Evaluation score (0-5)',
           x_axis_label="Course code",
           y_range=Range1d(2, 5),
           x_range=df_e["Course code"].unique().tolist(),
          width=1000,
          height =600)

# Create individual scatterplots
p.scatter(x="Course code", y='Evaluation', source=managementAndBuisness_source, legend_label="Management and Business",
          size=8, color="blue", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=medtec_source, legend_label="Medtech",
          size=8, color="green", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=manufacturingAndMaterials_source, legend_label="Manufacturing and Materials",
          size=8, color="red", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=designforpeople_source, legend_label="Design for People",
          size=8, color="purple", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=supplychain_source, legend_label="Supply Chain",
          size=8, color="orange", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=productdesign_source, legend_label="Product Design",
          size=8, color="purple", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=systemsengineering_source, legend_label="Systems Engineering",
          size=8, color="cyan", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=circularitySustainability_source, legend_label="Circularity and Sustainability",
          size=8, color="pink", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=SmartProducts_source, legend_label="Smart Products",
          size=8, color="brown", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=tecnicalCompetences_source, legend_label="Technical Competences",
          size=8, color="gray", alpha=0.6, marker="circle")

layout = column(p)

#Preactivate wheel zoom
wheel_zoom_tool = WheelZoomTool()
p.add_tools(wheel_zoom_tool)
p.toolbar.active_scroll = wheel_zoom_tool

# Plot style
p.xaxis.visible = True
p.legend.title = 'Learning categories'
p.legend.location = 'bottom_right'
p.legend.visible = True
p.legend.click_policy = 'hide'  # Clicking a legend item hides the corresponding data
p.xaxis.major_label_orientation = -0.785
#p.legend.spacing = 4

# Finalization
output_file("Evaluation_plot.html")
show(layout)
save(p)


'C:\\Users\\johan\\Final Project\\_Data load\\Evaluation_plot.html'

## Tap tool (fucking around)
Implement more stuff to show when you click
Assign legends as in assignment 2

In [26]:
#WORKS
from bokeh.models import ColumnDataSource, Range1d, WheelZoomTool, Div, CustomJS
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import column
import pandas as pd

# Create ColumnDataSources
all_source = ColumnDataSource(df_e.sort_values(by=["Evaluation"]))
#managementAndBuisness_source = ColumnDataSource(df_e[df_e['Category'].str.contains('managementAndBuisness', na=False)])
#medtec_source = ColumnDataSource(df_e[df_e['Category'].str.contains('medtec', na=False)])
#manufacturingAndMaterials_source = ColumnDataSource(df_e[df_e['Category'].str.contains('manufacturingAndMaterials', na=False)])
#designforpeople_source = ColumnDataSource(df_e[df_e['Category'].str.contains('designforpeople', na=False)])
#supplychain_source = ColumnDataSource(df_e[df_e['Category'].str.contains('supplychain', na=False)])
#productdesign_source = ColumnDataSource(df_e[df_e['Category'].str.contains('productdesign', na=False)])
#systemsengineering_source = ColumnDataSource(df_e[df_e['Category'].str.contains('systemsengineering', na=False)])
#circularitySustainability_source = ColumnDataSource(df_e[df_e['Category'].str.contains('circularitySustainability', na=False)])
#SmartProducts_source = ColumnDataSource(df_e[df_e['Category'].str.contains('SmartProducts', na=False)])
#tecnicalCompetences_source = ColumnDataSource(df_e[df_e['Category'].str.contains('tecnicalCompetences', na=False)])

# Create figure
p = figure(title="Course Evaluation Tool",
           y_axis_label='Evaluation score (0-5)',
           x_axis_label="Course code",
           y_range=Range1d(2, 5),
           x_range=sorted(df_e["Course code"].unique().tolist()),
           width=1000,
           height=600,
           tools="tap")

# Scatter plots for each category
#p.scatter(x="Course code", y='Evaluation', source=managementAndBuisness_source, legend_label="Management and Business", size=8, color="blue", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=medtec_source, legend_label="Medtech", size=8, color="green", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=manufacturingAndMaterials_source, legend_label="Manufacturing and Materials", size=8, color="red", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=designforpeople_source, legend_label="Design for People",size=8, color="purple", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=supplychain_source, legend_label="Supply Chain",size=8, color="orange", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=productdesign_source, legend_label="Product Design",size=8, color="purple", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=systemsengineering_source, legend_label="Systems Engineering",size=8, color="cyan", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=circularitySustainability_source, legend_label="Circularity and Sustainability",size=8, color="pink", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=SmartProducts_source, legend_label="Smart Products",size=8, color="brown", alpha=0.6, marker="circle")
#p.scatter(x='Course code', y='Evaluation', source=tecnicalCompetences_source, legend_label="Technical Competences",size=8, color="gray", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=all_source, legend_label="All",size=8, color="gray", alpha=0.6, marker="circle")
# Styling
p.legend.title = 'Learning categories'
p.legend.location = 'bottom_right'
p.legend.visible = True
p.legend.click_policy = 'hide'
p.xaxis.major_label_orientation = -0.785
p.xaxis.visible = True

# Zoom tool setup
wheel_zoom_tool = WheelZoomTool()
p.add_tools(wheel_zoom_tool)
p.toolbar.active_scroll = wheel_zoom_tool

# Div to display course info
div = Div(text="Click on a course dot to see details", width=800)

# JS callback on full source
callback = CustomJS(args=dict(source=all_source, div=div), code="""
    const indices = source.selected.indices;
    if (indices.length > 0) {
        const i = indices[0];
        const course = source.data['Course code'][i];
        const score = source.data['Evaluation'][i];
        const category = source.data['Category'][i];
        div.text = `<b>Course:</b> ${course} <br>
                    <b>Evaluation:</b> ${score} <br>
                    <b>Category:</b> ${category}`;
    }
""")
all_source.selected.js_on_change('indices', callback)

# Final layout and display
layout = column(p, div)

output_file("Evaluation_plot.html")
show(layout)


In [27]:
#FUCKING AROUND

In [28]:
unique_categories_string = ", ".join(df_e['Category'].unique().astype(str))


In [29]:
#WORKS
from bokeh.models import ColumnDataSource, Range1d, WheelZoomTool, Div, CustomJS
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import column
import pandas as pd

# Create ColumnDataSources
#all_source = ColumnDataSource(df_e.sort_values(by=["Evaluation"]))
managementAndBuisness_source = ColumnDataSource(df_e[df_e['Category'].str.contains('managementAndBuisness', na=False)])
medtec_source = ColumnDataSource(df_e[df_e['Category'].str.contains('medtec', na=False)])
manufacturingAndMaterials_source = ColumnDataSource(df_e[df_e['Category'].str.contains('manufacturingAndMaterials', na=False)])
designforpeople_source = ColumnDataSource(df_e[df_e['Category'].str.contains('designforpeople', na=False)])
supplychain_source = ColumnDataSource(df_e[df_e['Category'].str.contains('supplychain', na=False)])
productdesign_source = ColumnDataSource(df_e[df_e['Category'].str.contains('productdesign', na=False)])
systemsengineering_source = ColumnDataSource(df_e[df_e['Category'].str.contains('systemsengineering', na=False)])
circularitySustainability_source = ColumnDataSource(df_e[df_e['Category'].str.contains('circularitySustainability', na=False)])
SmartProducts_source = ColumnDataSource(df_e[df_e['Category'].str.contains('SmartProducts', na=False)])
tecnicalCompetences_source = ColumnDataSource(df_e[df_e['Category'].str.contains('tecnicalCompetences', na=False)])

# Create figure
p = figure(title="Course Evaluation Tool",
           y_axis_label='Evaluation score (0-5)',
           x_axis_label="Course code",
           y_range=Range1d(2, 5),
           x_range=sorted(df_e["Course code"].unique().tolist()),
           width=1000,
           height=600,
           tools="tap")

# Scatter plots for each category
#p.scatter(x='Course code', y='Evaluation', source=all_source, legend_label="All",size=8, color="gray", alpha=0.6, marker="circle")
p.scatter(x="Course code", y='Evaluation', source=managementAndBuisness_source, legend_label="Management and Business", size=8, color="blue", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=medtec_source, legend_label="Medtech", size=8, color="green", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=manufacturingAndMaterials_source, legend_label="Manufacturing and Materials", size=8, color="red", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=designforpeople_source, legend_label="Design for People",size=8, color="purple", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=supplychain_source, legend_label="Supply Chain",size=8, color="orange", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=productdesign_source, legend_label="Product Design",size=8, color="purple", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=systemsengineering_source, legend_label="Systems Engineering",size=8, color="cyan", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=circularitySustainability_source, legend_label="Circularity and Sustainability",size=8, color="pink", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=SmartProducts_source, legend_label="Smart Products",size=8, color="brown", alpha=0.6, marker="circle")
p.scatter(x='Course code', y='Evaluation', source=tecnicalCompetences_source, legend_label="Technical Competences",size=8, color="gray", alpha=0.6, marker="circle")


# Styling
p.legend.title = 'Learning categories'
p.legend.location = 'bottom_right'
p.legend.visible = True
p.legend.click_policy = 'mute'
p.xaxis.major_label_orientation = -0.785
p.xaxis.visible = True

# Zoom tool setup
wheel_zoom_tool = WheelZoomTool()
p.add_tools(wheel_zoom_tool)
p.toolbar.active_scroll = wheel_zoom_tool

# Div to display course info
div = Div(text="Click on a course dot to see details", width=800)

# Callbacks
callback1 = CustomJS(args=dict(source=managementAndBuisness_source, div=div), code="""
    const indices = source.selected.indices;
    if (indices.length > 0) {
        const i = indices[0];
        const course = source.data['Course code'][i];
        const score = source.data['Evaluation'][i];
        const category = source.data['Category'][i];
        div.text = `<b>Course:</b> ${course} <br>
                    <b>Evaluation:</b> ${score} <br>
                    <b>Category:</b> ${category}`;
    }
""")

callback2 = CustomJS(args=dict(source=medtec_source, div=div), code="""
    const indices = source.selected.indices;
    if (indices.length > 0) {
        const i = indices[0];
        const course = source.data['Course code'][i];
        const score = source.data['Evaluation'][i];
        const category = source.data['Category'][i];
        div.text = `<b>Course:</b> ${course} <br>
                    <b>Evaluation:</b> ${score} <br>
                    <b>Category:</b> ${category}`;
    }
""")

managementAndBuisness_source.selected.js_on_change('indices', callback1)
medtec_source.selected.js_on_change('indices', callback2)

# Final layout and display
layout = column(p, div)

output_file("Evaluation_plot.html")
show(layout)

In [30]:
#WORKS
from bokeh.models import ColumnDataSource, Range1d, WheelZoomTool, Div, CustomJS
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import column
import pandas as pd

# Create ColumnDataSources
all_source = ColumnDataSource(df_e.sort_values(by=["Evaluation"]))

# Create figure
p = figure(title="Course Evaluation Tool",
           y_axis_label='Evaluation score (0-5)',
           x_axis_label="Course code",
           y_range=Range1d(2, 5),
           x_range=sorted(df_e["Course code"].unique().tolist()),
           width=1000,
           height=600,
           tools="tap")

# Scatter plots for each category
p.scatter(x='Course code', y='Evaluation', source=all_source, legend_label="All",size=8, color="red", alpha=0.6, marker="circle")


# Adjust title font size and style
p.title.text_font_size = "20pt"  # Increase title font size
p.title.text_font_style = "bold"  # Make title bold
p.title.text_font = "Arial"  # Change title font (optional)

# Adjust axis labels font size and style
p.xaxis.axis_label_text_font_size = "14pt"  # X-axis label size
p.yaxis.axis_label_text_font_size = "14pt"  # Y-axis label size
p.xaxis.axis_label_text_font_style = "normal"  # X-axis label style
p.yaxis.axis_label_text_font_style = "normal"  # Y-axis label style
p.xaxis.axis_label_text_font = "Arial"  # X-axis label font (optional)
p.yaxis.axis_label_text_font = "Arial"  # Y-axis label font (optional)

# Adjust tick labels font size
p.xaxis.major_label_text_font_size = "12pt"  # X-axis tick labels
p.yaxis.major_label_text_font_size = "12pt"  # Y-axis tick labels



# Legend
#p.legend.title = 'Learning categories'
#p.legend.location = 'bottom_right'
#p.legend.visible = True
#p.legend.click_policy = 'mute'
#p.xaxis.major_label_orientation = -0.785
#p.xaxis.visible = True

# Zoom tool setup
wheel_zoom_tool = WheelZoomTool()
p.add_tools(wheel_zoom_tool)
p.toolbar.active_scroll = wheel_zoom_tool

# Div to display course info
div = Div(text="Click on a course dot to see details", width=800)

# Callbacks
callback = CustomJS(args=dict(source=all_source, div=div), code="""
    const indices = source.selected.indices;
    if (indices.length > 0) {
        const i = indices[0];
        const course = source.data['Course code'][i];
        const score = source.data['Evaluation'][i];
        const category = source.data['Category'][i];
        div.text = `<b>Course:</b> ${course} <br>
                    <b>Evaluation:</b> ${score} <br>
                    <b>Category:</b> ${category}`;
    }
""")

all_source.selected.js_on_change('indices', callback)

# Final layout and display
layout = column(p, div)

output_file("Evaluation_plot.html")
show(layout)